In [1]:
import os
import rasterio
import numpy as np
from tqdm import tqdm

In [2]:
def load_data(paths):
    # Get a list of all files in the directory (X_train, y_train, X_test)
    file_list = [[file for file in os.listdir(paths) if file.endswith('.tif')] for paths in directory_paths]

    # Sort the file list to ensure consistent order
    file_list = [sorted(files) for files in file_list]

    file_list = [file_list[0][0:4000], file_list[1][0:4000]]

    # Initialize an empty array to store the image data
    X_train, y_train, X_test = [], [], []

    # Iterate through the selected files
    for X_train_name, y_train_name in tqdm(zip(file_list[0], file_list[1])):
        # Construct the full path to the file
        X_train_path = os.path.join(paths[0], X_train_name)
        y_train_path = os.path.join(paths[1], y_train_name)

        # Open the raster file using rasterio
        with rasterio.open(X_train_path) as src:
            # Read the entire image data as a NumPy array
            image_data = src.read()

            # Append the image data to the array
            X_train.append(image_data)

        # Open the raster file using rasterio
        with rasterio.open(y_train_path) as src:
            # Read the entire image data as a NumPy array
            image_data = src.read()

            # Append the image data to the array
            y_train.append(image_data)

    # Convert the list of arrays to a single NumPy array
    X_train = np.array(X_train)
    y_train = np.array(y_train)

    return X_train, y_train

In [3]:
directory_paths = ['../data/raw/train_satellite', '../data/raw/train_kelp']
os.path.abspath(directory_paths[0])
X_train, y_train = load_data(directory_paths)

print("Shape of the X_train array:", X_train.shape)
print("Shape of the y_train array:", y_train.shape)

0it [00:00, ?it/s]

c:\Competitions\q2-detect-kelp\venv\lib\site-packages\rasterio\__init__.py:317: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
4000it [01:35, 41.69it/s]


Shape of the X_train array: (4000, 7, 350, 350)
Shape of the y_train array: (4000, 1, 350, 350)


In [4]:
# Create new binary y_train array of shape (n_images, 1) where 1 indicates kelp and 0 indicates no kelp in the image
y_train_binary = np.zeros((y_train.shape[0], 1))
for i in range(y_train.shape[0]):
    if np.sum(y_train[i]) > 0:
        y_train_binary[i] = 1


In [5]:
y_train_binary = y_train_binary.squeeze()

In [6]:
# Count the number of images with kelp and without kelp
print("Number of images with kelp:", np.sum(y_train_binary))

Number of images with kelp: 2495.0


In [7]:
def findConv2dOutShape(hin,win,conv,pool=2):
    # get conv arguments
    kernel_size=conv.kernel_size
    stride=conv.stride
    padding=conv.padding
    dilation=conv.dilation

    hout=np.floor((hin+2*padding[0]-dilation[0]*(kernel_size[0]-1)-1)/stride[0]+1)
    wout=np.floor((win+2*padding[1]-dilation[1]*(kernel_size[1]-1)-1)/stride[1]+1)

    if pool:
        hout/=pool
        wout/=pool
    return int(hout),int(wout)

import torch.nn as nn
import torch.nn.functional as F

# Neural Network
class Network(nn.Module):
    
    # Network Initialisation
    def __init__(self, params):
        
        super(Network, self).__init__()
    
        Cin,Hin,Win=params["shape_in"]
        init_f=params["initial_filters"] 
        num_fc1=params["num_fc1"]  
        num_classes=params["num_classes"] 
        self.dropout_rate=params["dropout_rate"] 
        
        # Convolution Layers
        self.conv1 = nn.Conv2d(Cin, init_f, kernel_size=3)
        h,w=findConv2dOutShape(Hin,Win,self.conv1)
        self.conv2 = nn.Conv2d(init_f, 2*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv2)
        self.conv3 = nn.Conv2d(2*init_f, 4*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv3)
        self.conv4 = nn.Conv2d(4*init_f, 8*init_f, kernel_size=3)
        h,w=findConv2dOutShape(h,w,self.conv4)
        
        # compute the flatten size
        self.num_flatten=h*w*8*init_f
        self.fc1 = nn.Linear(self.num_flatten, num_fc1)
        self.fc2 = nn.Linear(num_fc1, num_classes)

    def forward(self,X):
        
        # Convolution & Pool Layers
        X = F.relu(self.conv1(X)); 
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv3(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv4(X))
        X = F.max_pool2d(X, 2, 2)

        X = X.view(-1, self.num_flatten)
        
        X = F.relu(self.fc1(X))
        X=F.dropout(X, self.dropout_rate)
        X = self.fc2(X)
        return F.log_softmax(X, dim=1)

In [8]:
# Define a custom PyTorch dataset
from torch.utils.data import Dataset
class SatelliteDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        image = self.X[idx]
        label = self.y[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [9]:
# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train_binary, test_size=0.2, random_state=42)

# Find balance of kelp in y_train_split and y_test_split
print("Number of images with kelp in y_train_split:", np.sum(y_train_split))
print("Number of images with kelp in y_test_split:", np.sum(y_test_split))

Number of images with kelp in y_train_split: 1999.0
Number of images with kelp in y_test_split: 496.0


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Define the training parameters
params_train = {'batch_size': 64,
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': 64,
                'shuffle': True,
                'num_workers': 0}

# Define the training and validation sets
X_train = X_train / 4000
X_train = X_train.astype('float32')

# Split the data into training and validation sets
from sklearn.model_selection import train_test_split
X_train_split, X_test_split, y_train_split, y_test_split = train_test_split(X_train, y_train_binary, test_size=0.2, random_state=42)

# Create the training and validation datasets
dataset_train = SatelliteDataset(X_train_split, y_train_split)
dataset_val = SatelliteDataset(X_test_split, y_test_split)

# Create the training and validation generators
training_generator = DataLoader(dataset_train, **params_train)
validation_generator = DataLoader(dataset_val, **params_val)

# Define the model parameters
model_params = {
    "shape_in": (7, 350, 350),
    "initial_filters": 8,
    "num_fc1": 100,
    "num_classes": 2,
    "dropout_rate": 0.25
}

# Initialize the neural network
model = Network(model_params)

# Define the loss function and the optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Run the training loop
epochs = 100

for epoch in range(epochs):
    running_loss = 0.0
    running_corrects = 0
    model.train()

    for inputs, labels in training_generator:
        #inputs = inputs.to(device)
        #labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())
        loss.backward()
        optimizer.step()

        # Calculate the statistics
        _, preds = torch.max(outputs.data, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.long().data)

    epoch_loss = running_loss / len(dataset_train)
    epoch_acc = running_corrects.double() / len(dataset_train)

    print('Training Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

    # switch to evaluation mode
    model.eval()
    running_loss = 0.0
    running_corrects = 0

    for inputs, labels in validation_generator:
        #inputs = inputs.to(device)
        #labels = labels.to(device)

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels.long())

        # Calculate the statistics
        _, preds = torch.max(outputs.data, 1)
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.long().data)

    epoch_loss = running_loss / len(dataset_val)
    epoch_acc = running_corrects.double() / len(dataset_val)

    print('Validation Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))

Training Loss: 0.6588 Acc: 0.6494
Validation Loss: 0.6270 Acc: 0.6875
Training Loss: 0.6116 Acc: 0.6809
Validation Loss: 0.5938 Acc: 0.6987
Training Loss: 0.6030 Acc: 0.6878
Validation Loss: 0.5986 Acc: 0.6887
Training Loss: 0.5973 Acc: 0.6959
Validation Loss: 0.5882 Acc: 0.7000
Training Loss: 0.5865 Acc: 0.7016
Validation Loss: 0.5768 Acc: 0.7037
Training Loss: 0.5768 Acc: 0.7134
Validation Loss: 0.5768 Acc: 0.7087
Training Loss: 0.5772 Acc: 0.7034
Validation Loss: 0.5826 Acc: 0.6963
Training Loss: 0.5626 Acc: 0.7203
Validation Loss: 0.5912 Acc: 0.6937
Training Loss: 0.5520 Acc: 0.7191
Validation Loss: 0.5887 Acc: 0.7025
Training Loss: 0.5357 Acc: 0.7234
Validation Loss: 0.6088 Acc: 0.7000
Training Loss: 0.5653 Acc: 0.7206
Validation Loss: 0.6174 Acc: 0.6925
Training Loss: 0.5134 Acc: 0.7394
Validation Loss: 0.6627 Acc: 0.6850
Training Loss: 0.5182 Acc: 0.7356
Validation Loss: 0.6215 Acc: 0.7013
Training Loss: 0.4717 Acc: 0.7506
Validation Loss: 0.6975 Acc: 0.6813
Training Loss: 0.423

KeyboardInterrupt: 